In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
zai_api_key = os.getenv('Z_AI_API_KEY')

if zai_api_key:
    print(f"ZAI API Key exists and begins {zai_api_key[:8]}")
else:
    print("ZAI API Key not set")

ZAI API Key not set


In [3]:
# Initialize

zai_url="https://api.z.ai/api/coding/paas/v4"

zai = OpenAI(api_key=zai_api_key, base_url=zai_url)
MODEL = 'glm-4.5-air'

In [4]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [5]:
def chat(message, history):
    return "bananas"

In [6]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7905
* To create a public link, set `share=True` in `launch()`.


In [7]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [8]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7906
* To create a public link, set `share=True` in `launch()`.


## OK! Let's write a slightly better chat callback!

In [9]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = zai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7907
* To create a public link, set `share=True` in `launch()`.


In [11]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = zai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.


## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [13]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7909
* To create a public link, set `share=True` in `launch()`.


In [15]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7910
* To create a public link, set `share=True` in `launch()`.


In [17]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = zai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [18]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7911
* To create a public link, set `share=True` in `launch()`.
